# Case. Impact of Agent satisfaction sensibility

Situation:

    More agent availability on week days as weekends, 40% agents work on weekends, 60% not.
    
Task:

    Evaluate Shift Coverage over Agent satisfaction penalty for under-assignment
    Evaluate Agent Satisfaction over Agent satisfaction penalty for under-assignment using default function and shift stability function

In [1]:
import abm_scheduling
from abm_scheduling import Schedule as Schedule
from abm_scheduling import Nurse as Nurse

In [2]:
import time
from datetime import datetime

import abm_scheduling.Log
from abm_scheduling.Log import Log as Log

log = Log()

## Randomized. Define situation

In [3]:
p_to_accept_negative_change = .001
num_nurses_per_shift = 4
beta = 0.8

In [4]:
# 25 nurses, only 10 can work on weekends
nurses = []
for n in range(10):
    nurse = Nurse(id_name=n)
    nurse.generate_shift_preferences(degree_of_agent_availability=0.7, works_weekends=True)
    nurses.append(nurse)

In [5]:
nurses[1].print_shift_preferences()

Nurse 1's Preferences. Availability: 13(0.7) Min/Max: 6 / 6
+---------+----+----+----+----+----+----+----+
|         | Mo | Tu | We | Th | Fr | Sa | So |
+---------+----+----+----+----+----+----+----+
| shift 1 | x  | x  | x  | x  |    |    |    |
| shift 2 | x  | x  |    |    | x  | x  |    |
| shift 3 | x  |    |    | x  | x  | x  | x  |
+---------+----+----+----+----+----+----+----+


In [6]:
for n in range(15):
    nurse = Nurse(id_name=(10+n))
    nurse.generate_shift_preferences(degree_of_agent_availability=0.7, works_weekends=False)
    nurses.append(nurse)

In [7]:
schedule = Schedule(num_nurses_needed=num_nurses_per_shift, is_random=True)
model = abm_scheduling.NSP_AB_Model()
schedule.print_schedule(schedule_name="Intial Situation")

Week's Schedule Intial Situation
+---------+----------+----------+----------+----------+----------+----------+----------+
|         | Mo       | Tu       | We       | Th       | Fr       | Sa       | So       |
+---------+----------+----------+----------+----------+----------+----------+----------+
| shift 1 | need: 4  | need: 3  | need: 5  | need: 5  | need: 4  | need: 6  | need: 4  |
|         | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  |
| shift 2 | need: 5  | need: 4  | need: 5  | need: 4  | need: 7  | need: 4  | need: 7  |
|         | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  |
| shift 3 | need: 5  | need: 4  | need: 5  | need: 4  | need: 3  | need: 5  | need: 5  |
|         | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  |
+---------+----------+----------+----------+----------+----------+----------+----------+


In [16]:
model.show_hypothetical_max_schedule(schedule=schedule, nurses=nurses, print_detail_schedule = False, print_nurse_productivity = True)

Crude hypothetical shift coverage: 0.9795918367346939
Shift Coverage Hypothetical Maximum
+---------+------------+------------+------------+------------+------------+-----------+-----------+
|         | Mo         | Tu         | We         | Th         | Fr         | Sa        | So        |
+---------+------------+------------+------------+------------+------------+-----------+-----------+
| shift 1 | need: 4    | need: 3    | need: 5    | need: 5    | need: 4    | need: 6   | need: 4   |
|         | nurses: 17 | nurses: 19 | nurses: 18 | nurses: 16 | nurses: 18 | nurses: 5 | nurses: 7 |
|         | (4.25)     | (6.33)     | (3.6)      | (3.2)      | (4.5)      | (0.833)   | (1.75)    |
| shift 2 | need: 5    | need: 4    | need: 5    | need: 4    | need: 7    | need: 4   | need: 7   |
|         | nurses: 19 | nurses: 18 | nurses: 20 | nurses: 20 | nurses: 17 | nurses: 7 | nurses: 6 |
|         | (3.8)      | (4.5)      | (4)        | (5)        | (2.43)     | (1.75)    | (0.857)   |
|

In [9]:
model.get_total_agent_satisfaction(nurses)

-25000.0

In [10]:
# Default utility function (no agent satisfaction)
start_time = datetime.today()
results = model.run(schedule_org=schedule, nurses_org=nurses, beta=0.8, 
                         p_to_accept_negative_change=.001, utility_function_parameters = None)
end_time = datetime.today()
log.print_elapsed_time(start_time,end_time)

Week's Schedule Best Schedule. Beta: (0.8),p: (0.001)
+---------+------------+------------+-------------+------------+-------------+----------+----------+
|         | Mo         | Tu         | We          | Th         | Fr          | Sa       | So       |
+---------+------------+------------+-------------+------------+-------------+----------+----------+
| shift 1 | need: 4    | need: 3    | need: 5     | need: 5    | need: 4     | need: 6  | need: 4  |
|         | nurses:    | nurses:    | nurses:     | nurses:    | nurses:     | nurses:  | nurses:  |
|         | 12,0,3,17  | 0,6,24     | 10,16,19,6, | 24,3,19,8, | 23,3,24,4   | 8,9,4    | 9,2,3,0  |
|         |            |            | 18          | 7          |             |          |          |
| shift 2 | need: 5    | need: 4    | need: 5     | need: 4    | need: 7     | need: 4  | need: 7  |
|         | nurses:    | nurses:    | nurses:     | nurses:    | nurses:     | nurses:  | nurses:  |
|         | 2,7,5,24,  | 12,19,15,3 |

In [11]:
model.print_nurse_productivity(results.nurses)

Nurse productivity -  
Nr:   0, 	assig:6,	min:6,	max: 6,	deg.av:0.70,	prod: 1.00,	satisf:    768.60,	ok
Nr:   1, 	assig:5,	min:6,	max: 6,	deg.av:0.70,	prod: 0.83,	satisf:    533.67,	underbooked
Nr:   2, 	assig:4,	min:6,	max: 6,	deg.av:0.70,	prod: 0.67,	satisf:    281.67,	underbooked
Nr:   3, 	assig:4,	min:6,	max: 6,	deg.av:0.70,	prod: 0.67,	satisf:    281.67,	underbooked
Nr:   4, 	assig:6,	min:6,	max: 6,	deg.av:0.70,	prod: 1.00,	satisf:    768.60,	ok
Nr:   5, 	assig:4,	min:6,	max: 6,	deg.av:0.70,	prod: 0.67,	satisf:    281.67,	underbooked
Nr:   6, 	assig:4,	min:6,	max: 6,	deg.av:0.70,	prod: 0.67,	satisf:    281.67,	underbooked
Nr:   7, 	assig:5,	min:6,	max: 6,	deg.av:0.70,	prod: 0.83,	satisf:    533.67,	underbooked
Nr:   8, 	assig:3,	min:6,	max: 6,	deg.av:0.70,	prod: 0.50,	satisf:      8.33,	underbooked
Nr:   9, 	assig:4,	min:6,	max: 6,	deg.av:0.70,	prod: 0.67,	satisf:    281.67,	underbooked
Nr:  10, 	assig:2,	min:6,	max: 6,	deg.av:0.70,	prod: 0.33,	satisf:   -291.67,	underbooked
Nr:  

In [14]:
# Agent satisfaction utility function
utility_function_parameters = abm_scheduling.Utility_Function_Parameters()
utility_function_parameters.utility_function = 'agent_satisfaction'
results = model.run(schedule_org=schedule, nurses_org=nurses, 
                         p_to_accept_negative_change=.001, utility_function_parameters = utility_function_parameters)
end_time = datetime.today()
log.print_elapsed_time(start_time,end_time)

Week's Schedule Best Schedule. Beta: (0.8),p: (0.001)
+---------+--------------+------------+-------------+-------------+--------------+----------+----------+
|         | Mo           | Tu         | We          | Th          | Fr           | Sa       | So       |
+---------+--------------+------------+-------------+-------------+--------------+----------+----------+
| shift 1 | need: 4      | need: 3    | need: 5     | need: 5     | need: 4      | need: 6  | need: 4  |
|         | nurses:      | nurses:    | nurses:     | nurses:     | nurses:      | nurses:  | nurses:  |
|         | 6,19,17,13   | 13,7,12    | 14,17,6,9,  | 7,8,3,17,   | 0,11,23,9    | 9,4,2    | 9,0,3    |
|         |              |            | 13          | 0           |              |          |          |
| shift 2 | need: 5      | need: 4    | need: 5     | need: 4     | need: 7      | need: 4  | need: 7  |
|         | nurses:      | nurses:    | nurses:     | nurses:     | nurses:      | nurses:  | nurses:  |
|

In [15]:
model.print_nurse_productivity(results.nurses)

Nurse productivity -  
Nr:   0, 	assig:4,	min:6,	max: 6,	deg.av:0.70,	prod: 0.67,	satisf:    281.67,	underbooked
Nr:   1, 	assig:5,	min:6,	max: 6,	deg.av:0.70,	prod: 0.83,	satisf:    533.67,	underbooked
Nr:   2, 	assig:3,	min:6,	max: 6,	deg.av:0.70,	prod: 0.50,	satisf:      8.33,	underbooked
Nr:   3, 	assig:4,	min:6,	max: 6,	deg.av:0.70,	prod: 0.67,	satisf:    281.67,	underbooked
Nr:   4, 	assig:5,	min:6,	max: 6,	deg.av:0.70,	prod: 0.83,	satisf:    533.67,	underbooked
Nr:   5, 	assig:5,	min:6,	max: 6,	deg.av:0.70,	prod: 0.83,	satisf:    533.67,	underbooked
Nr:   6, 	assig:5,	min:6,	max: 6,	deg.av:0.70,	prod: 0.83,	satisf:    533.67,	underbooked
Nr:   7, 	assig:4,	min:6,	max: 6,	deg.av:0.70,	prod: 0.67,	satisf:    281.67,	underbooked
Nr:   8, 	assig:6,	min:6,	max: 6,	deg.av:0.70,	prod: 1.00,	satisf:    768.60,	ok
Nr:   9, 	assig:4,	min:6,	max: 6,	deg.av:0.70,	prod: 0.67,	satisf:    281.67,	underbooked
Nr:  10, 	assig:3,	min:6,	max: 6,	deg.av:0.70,	prod: 0.50,	satisf:      8.33,	underboo

## Targeted search. Define situation

In [ ]:
#%% Initializations
p_to_accept_negative_change = .001

# Situation definition
matrix_nurses_needed = [5,6,3, 5,6,3, 5,6,3, 5,6,3, 5,6,5, 6,6,5, 6,6,3]
#type 1 only first shift work week days
matrix_nurse_availability_type1 = ['x','','', 'x','','',  'x','','', 'x','','', 'x','','', 'x','','', '','','x']
#type 2 1s+2s work week days
matrix_nurse_availability_type2 = ['x','x','', 'x','x','',  'x','x','', 'x','x','', 'x','x','', '','','', '','','']
#type 3 1s+2s+3s work week days
matrix_nurse_availability_type3 = ['x','x','x', 'x','x','x',  'x','x','x', 'x','x','x', 'x','x','', 'x','x','', 'x','x','']
#type 4 "springers"
matrix_nurse_availability_type4 = ['x','x','x', 'x','x','x',  'x','x','x', 'x','x','x', 'x','x','x', 'x','x','x', 'x','x','x']
maximum_shifts_type4=3
#type 5 work only weekends
matrix_nurse_availability_type5 = ['','','', '','','', '','','', '','','', '','','x', 'x','x','x', 'x','x','x']


# Create Schedule
#schedule_random = Schedule(matrix_nurses_needed=matrix_nurses_needed, is_random=False)
schedule = Schedule(matrix_nurses_needed=matrix_nurses_needed)

# Create model and nurses
model = abm_scheduling.NSP_AB_Model()
#nurses = model.generate_nurses(10, 0.5, True)
list_nurse_schedules = []
list_nurse_schedules.append(matrix_nurse_availability_type1)
list_nurse_schedules.append(matrix_nurse_availability_type1)
list_nurse_schedules.append(matrix_nurse_availability_type1)
list_nurse_schedules.append(matrix_nurse_availability_type1)
list_nurse_schedules.append(matrix_nurse_availability_type2)
list_nurse_schedules.append(matrix_nurse_availability_type2)
list_nurse_schedules.append(matrix_nurse_availability_type2)
list_nurse_schedules.append(matrix_nurse_availability_type2)
list_nurse_schedules.append(matrix_nurse_availability_type3)
list_nurse_schedules.append(matrix_nurse_availability_type3)
list_nurse_schedules.append(matrix_nurse_availability_type4)
list_nurse_schedules.append(matrix_nurse_availability_type4)
list_nurse_schedules.append(matrix_nurse_availability_type5)
list_nurse_schedules.append(matrix_nurse_availability_type5)
list_nurse_schedules.append(matrix_nurse_availability_type5)

nurses = model.generate_nurses_from_nurse_schedules(list_nurse_schedules)
schedule.print_schedule(schedule_name="Intial Situation")

In [ ]:
nurses[10].minimum_shifts = 2
nurses[10].maximum_shifts = 4
nurses[10].print_shift_preferences()
nurses[11].minimum_shifts = 2
nurses[11].maximum_shifts = 4
nurses[11].print_shift_preferences()

In [ ]:
model.get_total_agent_satisfaction(nurses)

In [ ]:
model.print_nurse_productivity(nurses)

In [ ]:
model.show_hypothetical_max_schedule(schedule=schedule, nurses=nurses)

In [ ]:
# Default utility function (no agent satisfaction)
start_time = datetime.today()
results = model.run(schedule_org=schedule, nurses_org=nurses, beta=0.9, 
                         p_to_accept_negative_change=.001, utility_function_parameters = None)
end_time = datetime.today()
log.print_elapsed_time(start_time,end_time)

In [ ]:
model.print_nurse_productivity(results.nurses)

In [ ]:
# Agent satisfaction utility function
utility_function_parameters = abm_scheduling.Utility_Function_Parameters()
utility_function_parameters.utility_function = 'agent_satisfaction'
results = model.run(schedule_org=schedule, nurses_org=nurses, beta=0.9, 
                         p_to_accept_negative_change=.001, utility_function_parameters = utility_function_parameters)
end_time = datetime.today()
log.print_elapsed_time(start_time,end_time)

In [ ]:
model.print_nurse_productivity(results.nurses)